# STEP 1: Create Matching Engine Index and Endpoint for Retrieval

Configure parameters to create Matching Engine index
- ME_REGION: Region where Matching Engine Index and Index Endpoint are deployed
- ME_INDEX_NAME: Matching Engine index display name
- ME_EMBEDDING_DIR: Cloud Storage path to allow inserting, updating or deleting the contents of the Index
- STEP 1: Create Matching Engine Index and Endpoint for Retrieval
ME_DIMENSIONS: The number of dimensions of the input vectors. Vertex AI Embedding API generates 768 dimensional vector embeddings.

### Install packages

In [11]:
# Install Vertex AI LLM SDK
! pip install --user --upgrade google-cloud-aiplatform==1.35.0 langchain==0.0.323

# Dependencies required by Unstructured PDF loader
! sudo apt -y -qq install tesseract-ocr libtesseract-dev
! sudo apt-get -y -qq install poppler-utils
! pip install --user unstructured==0.7.5 pdf2image==1.16.3 pytesseract==0.3.10 pdfminer.six==20221105

# For Matching Engine integration dependencies (default embeddings)
! pip install --user tensorflow_hub==0.13.0 tensorflow_text==2.12.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.6 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
The following additional packages will be installed:
  libarchive-dev liblept5 libleptonica-dev libopenjp2-7 libtesseract4
  libwebpmux3 tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  libarchive-dev liblept5 libleptonica-dev libopenjp2-7 libtesseract-dev
  libtesseract4 libwebpmux3 tesseract-oc

### Restart current runtime
To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [13]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import os
import urllib.request

if not os.path.exists("utils"):
    os.makedirs("utils")

url_prefix = "https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/language/use-cases/document-qa/utils"
files = ["__init__.py", "matching_engine.py", "matching_engine_utils.py"]

for fname in files:
    urllib.request.urlretrieve(f"{url_prefix}/{fname}", filename=f"utils/{fname}")

In [3]:
import uuid
import json
import time
import numpy as np
import langchain
print(f"LangChain version: {langchain.__version__}")

from typing import List

from utils.matching_engine import MatchingEngine
from utils.matching_engine_utils import MatchingEngineUtils
from langchain.document_loaders import GCSDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import VertexAIEmbeddings

from pydantic import BaseModel

LangChain version: 0.0.323


In [4]:
PROJECT_ID = "engaged-domain-403109"  # @param {type:"string"}
REGION = "asia-southeast1"  # @param {type:"string"}

ME_REGION = REGION
ME_INDEX_NAME = f"{PROJECT_ID}-me-index"  # @param {type:"string"}
ME_EMBEDDING_DIR = f"{PROJECT_ID}-me-bucket"  # @param {type:"string"}
ME_DIMENSIONS = 768  # when using Vertex PaLM Embedding

## Make a Google Cloud Storage bucket for your Matching Engine index

In [3]:
! set -x && gsutil mb -p $PROJECT_ID -l $REGION gs://$ME_EMBEDDING_DIR

+ gsutil mb -p engaged-domain-403109 -l asia-southeast1 gs://engaged-domain-403109-me-bucket
Creating gs://engaged-domain-403109-me-bucket/...
ServiceException: 409 A Cloud Storage bucket named 'engaged-domain-403109-me-bucket' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


### Create a dummy embeddings file to initialize when creating the index

In [4]:
# dummy embedding
init_embedding = {"id": str(uuid.uuid4()), "embedding": list(np.zeros(ME_DIMENSIONS))}

# dump embedding to a local file
with open("embeddings_0.json", "w") as f:
    json.dump(init_embedding, f)

# write embedding to Cloud Storage
! set -x && gsutil cp embeddings_0.json gs://{ME_EMBEDDING_DIR}/init_index/embeddings_0.json

+ gsutil cp embeddings_0.json gs://engaged-domain-403109-me-bucket/init_index/embeddings_0.json
Copying file://embeddings_0.json [Content-Type=application/json]...
/ [1 files][  3.8 KiB/  3.8 KiB]                                                
Operation completed over 1 objects/3.8 KiB.                                      


## Create Index

In [12]:
mengine = MatchingEngineUtils(PROJECT_ID, ME_REGION, ME_INDEX_NAME)

In [2]:
index = mengine.create_index(
    embedding_gcs_uri=f"gs://{ME_EMBEDDING_DIR}/init_index",
    dimensions=ME_DIMENSIONS,
    index_update_method="streaming", # can change to batch updates if needed
    index_algorithm="tree-ah",
)
if index:
    print(index.name)

NameError: name 'mengine' is not defined

## Deploy Index to Endpoint
Deploy index to Index Endpoint on Matching Engine. This notebook deploys the index to a public endpoint. The deployment operation creates a public endpoint that will be used for querying the index for approximate nearest neighbors.

For deploying index to a Private Endpoint, refer to the documentation to set up pre-requisites.

In [6]:
index_endpoint = mengine.deploy_index()
if index_endpoint:
    print(f"Index endpoint resource name: {index_endpoint.name}")
    print(
        f"Index endpoint public domain name: {index_endpoint.public_endpoint_domain_name}"
    )
    print("Deployed indexes on the index endpoint:")
    for d in index_endpoint.deployed_indexes:
        print(f"    {d.id}")

INFO:root:Index endpoint engaged-domain-403109-me-index-endpoint does not exists. Creating index endpoint...
INFO:root:Deploying index to endpoint with long running operation projects/510519063638/locations/asia-southeast1/indexEndpoints/3617586769429528576/operations/5185750934893887488
INFO:root:Poll the operation to create index endpoint ...
INFO:root:Index endpoint engaged-domain-403109-me-index-endpoint created with resource name as projects/510519063638/locations/asia-southeast1/indexEndpoints/3617586769429528576 and endpoint domain name as 
INFO:root:Deploying index with request = {'id': 'engaged_domain_403109_me_index_20231213104529', 'display_name': 'engaged_domain_403109_me_index_20231213104529', 'index': 'projects/510519063638/locations/asia-southeast1/indexes/4693366538231611392', 'dedicated_resources': {'machine_spec': {'machine_type': 'e2-standard-2'}, 'min_replica_count': 2, 'max_replica_count': 10}}


.

INFO:root:Poll the operation to deploy index ...


........................

INFO:root:Deployed index engaged-domain-403109-me-index to endpoint engaged-domain-403109-me-index-endpoint


.Index endpoint resource name: projects/510519063638/locations/asia-southeast1/indexEndpoints/3617586769429528576
Index endpoint public domain name: 
Deployed indexes on the index endpoint:


# STEP 2: Add Document Embeddings to Matching Engine - Vector Store
This step ingests and parse PDF documents, split them, generate embeddings and add the embeddings to the vector store. The document corpus used as dataset is a sample of Google published research papers across different domains - large models, traffic simulation, productivity etc.

## Ingest PDF files
The document corpus is hosted on Cloud Storage bucket (at gs://github-repo/documents/google-research-pdfs/) and LangChain provides a convenient document loader GCSDirectoryLoader to load documents from a Cloud Storage bucket. The loader uses Unstructured package to load files of many types including pdfs, images, html and more.

Make a Google Cloud Storage bucket in your GCP project to copy the document files into.



In [ ]:
# ! set -x && gsutil mb -p $PROJECT_ID -l us-central1 gs://$GCS_BUCKET_DOCS

In [4]:
GCS_BUCKET_DOCS = "financial-websites-pdfs"
folder_prefix = "pdfs"

In [5]:
# Load documents and add document metadata such as file name, to be retrieved later when citing the references.

# Ingest PDF files

print(f"Processing documents from {GCS_BUCKET_DOCS}")
loader = GCSDirectoryLoader(
    project_name=PROJECT_ID, bucket=GCS_BUCKET_DOCS, prefix=folder_prefix
)
documents = loader.load()

# Add document name and source to the metadata
for document in documents:
    doc_md = document.metadata
    document_name = doc_md["source"].split("/")[-1]
    # derive doc source from Document loader
    doc_source_prefix = "/".join(GCS_BUCKET_DOCS.split("/")[:3])
    doc_source_suffix = "/".join(doc_md["source"].split("/")[4:-1])
    source = f"{doc_source_prefix}/{doc_source_suffix}"
    document.metadata = {"source": source, "document_name": document_name}

print(f"# of documents loaded (pre-chunking) = {len(documents)}")

Processing documents from financial-websites-pdfs


[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


# of documents loaded (pre-chunking) = 19


In [6]:
# Verify document metadata

documents[0].metadata


{'source': 'financial-websites-pdfs/',
 'document_name': 'annual value as of 2022 for social support schemes.pdf'}

## Chunk documents
Split the documents to smaller chunks. When splitting the document, ensure a few chunks can fit within the context length of LLM.

In [9]:
# split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
)
doc_splits = text_splitter.split_documents(documents)

# Add chunk number to metadata
for idx, split in enumerate(doc_splits):
    split.metadata["chunk"] = idx

print(f"# of documents = {len(doc_splits)}")

# of documents = 195


In [10]:
doc_splits[0].metadata


{'source': 'financial-websites-pdfs/',
 'document_name': 'annual value as of 2022 for social support schemes.pdf',
 'chunk': 0}

## Configure Matching Engine as Vector Store

### Get Matching Engine Index id and Endpoint id

In [13]:
ME_INDEX_ID, ME_INDEX_ENDPOINT_ID = mengine.get_index_and_endpoint()
print(f"ME_INDEX_ID={ME_INDEX_ID}")
print(f"ME_INDEX_ENDPOINT_ID={ME_INDEX_ENDPOINT_ID}")

ME_INDEX_ID=projects/510519063638/locations/asia-southeast1/indexes/4693366538231611392
ME_INDEX_ENDPOINT_ID=projects/510519063638/locations/asia-southeast1/indexEndpoints/3617586769429528576


### Next you will define some utility functions that you will use for the Vertex AI Embeddings API


In [5]:
# Utility functions for Embeddings API with rate limiting
def rate_limit(max_per_minute):
    period = 60 / max_per_minute
    print("Waiting")
    while True:
        before = time.time()
        yield
        after = time.time()
        elapsed = after - before
        sleep_time = max(0, period - elapsed)
        if sleep_time > 0:
            print(".", end="")
            time.sleep(sleep_time)


class CustomVertexAIEmbeddings(VertexAIEmbeddings, BaseModel):
    requests_per_minute: int
    num_instances_per_batch: int

    # Overriding embed_documents method
    def embed_documents(self, texts: List[str]):
        limiter = rate_limit(self.requests_per_minute)
        results = []
        docs = list(texts)

        while docs:
            # Working in batches because the API accepts maximum 5
            # documents per request to get embeddings
            head, docs = (
                docs[: self.num_instances_per_batch],
                docs[self.num_instances_per_batch :],
            )
            chunk = self.client.get_embeddings(head)
            results.extend(chunk)
            next(limiter)

        return [r.values for r in results]

### Initialize Matching Engine vector store with text embeddings model


In [6]:
# Embeddings API integrated with langChain
EMBEDDING_QPM = 100
EMBEDDING_NUM_BATCH = 5
embeddings = CustomVertexAIEmbeddings(
    requests_per_minute=EMBEDDING_QPM,
    num_instances_per_batch=EMBEDDING_NUM_BATCH,
)

# initialize vector store
me = MatchingEngine.from_components(
    project_id=PROJECT_ID,
    region=ME_REGION,
    gcs_bucket_name=f"gs://{ME_EMBEDDING_DIR}".split("/")[2],
    embedding=embeddings,
    index_id=ME_INDEX_ID,
    endpoint_id=ME_INDEX_ENDPOINT_ID,
)

2024-03-02 13:24:52.477684: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarni

NameError: name 'ME_INDEX_ID' is not defined

## Add documents as embeddings in Matching Engine as index

The document chunks are transformed as embeddings (vectors) using Vertex AI Embeddings API and added to the index with streaming index update. With Streaming Updates, you can update and query your index within a few seconds.

The original document text is stored on Cloud Storage bucket had referenced by id.

Prepare text and metadata to be added to the vectors

In [27]:
# Store docs as embeddings in Matching Engine index
# It may take a while since API is rate limited
texts = [doc.page_content for doc in doc_splits]
metadatas = [
    [
        {"namespace": "source", "allow_list": [doc.metadata["source"]]},
        {"namespace": "document_name", "allow_list": [doc.metadata["document_name"]]},
        {"namespace": "chunk", "allow_list": [str(doc.metadata["chunk"])]},
    ]
    for doc in doc_splits
]

### Add embeddings to the vector store

NOTE: Depending on the volume and size of documents, this step may take time.



In [30]:
doc_ids = me.add_texts(texts=texts, metadatas=metadatas)


Waiting
..................................

INFO:root:Indexed 195 documents to Matching Engine.


## Validate semantic search with Matching Engine is working



In [34]:
# Test whether search from vector store is working
me.similarity_search("What are the criterias to go to schools for SPED?", k=2)

Waiting


[Document(page_content="Special physical facilities which may include sensory modulation rooms, vocational training rooms, depending on the needs\n\nof their students\n\nConsiderations when selecting a SPED school:\n\n6 Additionally, in selecting a SPED school for your child, you can consider these other factors :\n\nYour child’s needs and education pathways\n\nDistance from home to school – A nearer school means reduced transport costs and shorter travelling time\n\nYour child’s interest and whether the school o\x00ers CCAs and activities that matches these interests; and\n\nSchool identity, including the school’s vision, mission, culture.\n\nDepending on your child's abilities, you can enrol your child into di\x00erent types of SPED schools:\n\nThose that follow the National Curriculum (e.g. Pathlight School)\n\nYour child will need to have adequate cognitive and adaptive skills to keep up with the mainstream curriculum Your child will receive support in daily living and social-emoti